In [ ]:
"""
@brief: After creating the “gold-standard” dataset, our task is to identify the most appropriate base language model that could be further fine-tuned on the dataset obtained in Step 1.

Since our text corpus primarily consists of scientific texts in the biomedical domain, potential candidates for our choice of the base model include:

BiomedNLP-PubMedBERT-base-uncased-abstract-full text
SciBERT
ElectraMed
"""

In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 582 kB 8.9 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 398 kB 74.0 MB/s 
     |████████████████████████████████| 136 kB 71.9 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 271 kB 73.3 MB/s 
     |████████████████████████████████| 144 kB 76.8 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

In [ ]:
!pip install transformers==4.10.1 --quiet

     |████████████████████████████████| 2.8 MB 7.9 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 3.3 MB 73.5 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 


In [ ]:
!pip install "torchmetrics<0.7"

     |████████████████████████████████| 332 kB 7.7 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.7.3
    Uninstalling torchmetrics-0.7.3:
      Successfully uninstalled torchmetrics-0.7.3


In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=cde3cad6e666b5a3a4975a38a7e672558d8930e4d14398deb19948cd768d0ae0
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
#testing gpu prescence
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import os
import itertools
import json
import srsly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
from tqdm.auto import tqdm

#pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler
from transformers import AutoTokenizer
from transformers import BertTokenizerFast as BertTokenizer, AutoModelForTokenClassification as AMTC, AdamW as Adam
from transformers import DataCollatorWithPadding

#padding sequences
from keras.preprocessing.sequence import pad_sequences

#pytorch scheduler
from transformers import get_linear_schedule_with_warmup

#pytorch lightning - lots of features like model checkpoints, logging, metrics, etc
import pytorch_lightning as pl
#checkpointing and saving models
from pytorch_lightning.callbacks import ModelCheckpoint
#from torchmetrics import Accuracy
from torchmetrics.functional import auroc 
#going to create a confusion matrix
from sklearn.metrics import classification_report, multilabel_confusion_matrix
#f1-score
from seqeval.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
import os
project_folder = "gdrive/MyDrive/Sanah/Data"
os.chdir(project_folder)

In [ ]:
!ls

checkpoints  ner_drug_disease_poster.jsonl  taggedData


In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines as jl
file_name = "ner_drug_disease_poster.jsonl"
spanData = []
with jl.open(file_name) as reader:
    for obj in reader:
      for key in list(obj.keys()):
        if key[0] == "_":
          obj.pop(key, None)
        spanData.append(obj)
      
#print(spanData[0]['text'])     

In [ ]:
#number of spans collected
print("Training data shape:", len(spanData))
trainingDataSize = len(spanData)

Training data shape: 5786


In [ ]:
"""
We will limit our sequence length to 75 tokens and we will use a batch size of 32 as suggested by the Bert paper.
"""

'\nWe will limit our sequence length to 75 tokens and we will use a batch size of 32 as suggested by the Bert paper.\n'

In [ ]:
MAX_LENGTH =  75#@param {type: "integer"}
task = "ner"
MODEL = "allenai/scibert_scivocab_uncased" 
OUTPUT_DIR = "bert-ner" #@param ["bert-ner"]
BATCH_SIZE = 32 #@param {type: "integer"}
NUM_EPOCHS = 3 #@param {type: "integer"}
SAVE_STEPS = 100 #@param {type: "integer"}
LOGGING_STEPS = 100 #@param {type: "integer"}
SEED = 42 #@param {type: "integer"}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
data_collator = DataCollatorWithPadding(tokenizer)


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

tokens_and_ner_tag = []
LABELS = []
POS_TAGS = []

for obj in spanData:
  tokens = obj["tokens"]
  spans = obj["spans"]
  for tok in tokens:
    filter_search = [spanDict['label'] for spanDict in spans if int(spanDict['token_start']) <= int(tok['id']) <= int(spanDict['token_end'])]
    if len(filter_search) > 0:
      tok['label'] = (filter_search[0])
    else:
      tok['label'] = "O"
    #extracting the pos tag
    tok['pos'] = nltk.pos_tag([tok['text']])[0][1]
    if tok['label'] not in LABELS:
      LABELS.append(tok['label'])
    if tok['pos'] not in POS_TAGS:
      POS_TAGS.append(tok['pos'])
#chunkify and add to tokens_and_ner_tag
#NEW ADDITION: IOB SCHEME for chunk breakdown
for obj in spanData:
  tokens = obj["tokens"]
  prevTag = None
  currTag = None
  currTok = None
  for tok in tokens:
    currTag = tok['label']
    if currTag in ["DRUG", "DISEASE"]:
      if currTag == prevTag:
        if "startChunk" not in currTok.keys():
          currTok['label'] = "B-" + currTag
          currTok['startChunk'] = True
        tok['label'] = "I-"+currTag
        #currTok['text'] += " " + tok['text']
        #tok['delete'] = True
      else:
        currTok = tok
        prevTag = currTag
    else:
      currTok = tok
      prevTag = currTag

for i,obj in enumerate(spanData):
  tokens = obj["tokens"]
  new_tokens = []
  for tok in tokens:
    tok_dict_new = {}
    if 'delete' not in tok:
      tok_dict_new['text_id'] = str(i)
      tok_dict_new['text'] = tok['text']
      tok_dict_new['label'] = tok['label']
      tok_dict_new['pos'] = tok['pos']
      new_tokens.append(tok_dict_new)
  tokens_and_ner_tag.append(new_tokens)

"""
we could also try and work from the sentence-level
for sent in nltk.sent_tokenize(sentence):
  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
     if hasattr(chunk, 'label'):
        print(chunk.label(), ' '.join(c[0] for c in chunk))
"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


"\nwe could also try and work from the sentence-level\nfor sent in nltk.sent_tokenize(sentence):\n  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):\n     if hasattr(chunk, 'label'):\n        print(chunk.label(), ' '.join(c[0] for c in chunk))\n"

In [ ]:
print(tokens_and_ner_tag[2])

[{'text_id': '2', 'text': '1', 'label': 'O', 'pos': 'CD'}, {'text_id': '2', 'text': 'University', 'label': 'O', 'pos': 'NNP'}, {'text_id': '2', 'text': 'of', 'label': 'O', 'pos': 'IN'}, {'text_id': '2', 'text': 'Turin', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'text': ',', 'label': 'O', 'pos': ','}, {'text_id': '2', 'text': 'Turin', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'text': ',', 'label': 'O', 'pos': ','}, {'text_id': '2', 'text': 'Italy', 'label': 'O', 'pos': 'NNP'}, {'text_id': '2', 'text': ';', 'label': 'O', 'pos': ':'}, {'text_id': '2', 'text': '2', 'label': 'O', 'pos': 'CD'}, {'text_id': '2', 'text': 'Fondazione', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'text': 'IRCCS', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'text': 'Ca', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'text': '’', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'text': 'Granda', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'text': 'Policlinico', 'label': 'O', 'pos': 'NN'}, {'text_id': '2', 'tex

In [ ]:
"""
from glob import glob
res = [f for f in glob("*.txt") if "sample" in f ]
print(len(res))
for file in res:
    os.remove(file)
"""

'\nfrom glob import glob\nres = [f for f in glob("*.txt") if "sample" in f ]\nprint(len(res))\nfor file in res:\n    os.remove(file)\n'

In [ ]:
!ls

checkpoints  ner_drug_disease_poster.jsonl  taggedData


In [ ]:
#saving my data in a special folder
"""
trainPath = "./taggedData/"
os.mkdir(trainPath)
root_file_name = "sample"
for i, row in enumerate(tokens_and_ner_tag):
  with open(trainPath + root_file_name + str(i) + '.txt', 'w') as f:
    for tok_dict in row:
      f.write(tok_dict['text_id'] + ", " + tok_dict['text'] + ", " + tok_dict['pos'] + ", " + tok_dict['label'])
      f.write("\n")
    f.close()
"""

'\ntrainPath = "./taggedData/"\nos.mkdir(trainPath)\nroot_file_name = "sample"\nfor i, row in enumerate(tokens_and_ner_tag):\n  with open(trainPath + root_file_name + str(i) + \'.txt\', \'w\') as f:\n    for tok_dict in row:\n      f.write(tok_dict[\'text_id\'] + ", " + tok_dict[\'text\'] + ", " + tok_dict[\'pos\'] + ", " + tok_dict[\'label\'])\n      f.write("\n")\n    f.close()\n'

In [ ]:
trainPath = "./taggedData/"

In [ ]:
LABELS.extend(["B-DISEASE", "I-DISEASE", "B-DRUG", "I-DRUG"])
#adding a padding label for both the ner labels and pos tags
LABELS.append("PAD")
POS_TAGS.append("PAD")

#ner tag and pos tag encodings
label_encoding_dict = {t: i for i, t in enumerate(LABELS)}
pos_encoding_dict = {t: i for i, t in enumerate(POS_TAGS)}

In [ ]:
#make dataset into this schema

def get_all_tokens_and_ner_tags(directory):
    pdFrames = [get_tokens_and_ner_tags(os.path.join(directory, filename)) for filename in os.listdir(directory) if filename.startswith("sample")]
    return pd.concat(pdFrames).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        text_ids = list(np.concatenate([[x.split(',')[0] for x in y] for y in split_list]).flat)
        tokens = list(np.concatenate([[x.split(',')[1] for x in y] for y in split_list]).flat)
        pos = list(np.concatenate([[x.split(',')[2] for x in y] for y in split_list]).flat)
        entities = list(np.concatenate([[x.split(',')[3].strip() for x in y] for y in split_list]).flat)
    mFrame =  pd.DataFrame({"sent_ids": text_ids, 'tokens': tokens, "pos": pos, 'ner_tags': entities})
    return mFrame
  
  
def get_med_token_dataset(train_directory):
    train_df = get_all_tokens_and_ner_tags(train_directory)
    return (train_df)

train_dataset = get_med_token_dataset(trainPath)
print(len(train_dataset))


188430


In [ ]:
train_dataset = train_dataset.drop_duplicates()
train_dataset = train_dataset.dropna()

In [ ]:
train_dataset.head(20)

,sent_ids,tokens,pos,ner_tags
0,4786,2,CD,O
1,4786,:,:,O
2,4786,Electron,NN,O
3,4786,Microscopy,NN,O
4,4786,and,CC,O
5,4786,gingipain,NN,O
6,4786,presence,NN,O
8,4786,activity,NN,O
9,4786,on,IN,O
10,4786,OMVs,NN,O


In [ ]:
train_dataset["ner_tags"].values

array(['O', 'O', 'O', ..., 'O', 'O', 'O'], dtype=object)

In [ ]:
"""
from sklearn.model_selection import train_test_split
traindf, valdf = train_test_split(train_dataset, test_size=0.2)
traindf.shape, valdf.shape
"""

'\nfrom sklearn.model_selection import train_test_split\ntraindf, valdf = train_test_split(train_dataset, test_size=0.2)\ntraindf.shape, valdf.shape\n'

# Trainer Pipeline

#Quick Modeling

In [ ]:
class SentenceGetter(object):
    def __init__(self, data):
        self.num_sent = 1
        self.data = data
        self.empty = False
       # agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
       #                                                    s["POS"].values.tolist(),
        #                                                   s["Tag"].values.tolist())]
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["tokens"].values.tolist(), s["pos"].values.tolist(),
                                                           s["ner_tags"].values.tolist())]
        self.grouped = self.data.groupby("sent_ids").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["{}".format(self.num_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(train_dataset)

NameError: ignored

In [ ]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
pos_tags = [[s[1] for s in sentence] for sentence in getter.sentences]
ner_tags = [[s[2] for s in sentence] for sentence in getter.sentences]

In [ ]:
#tokenization function
def tokenize_and_align_labels(examples, pos_t, ner_tags):
    tokenized_sentence = []
    labels = []
    pos_tags = []
    for word, pos, label in zip(examples,pos_t, ner_tags):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        #Add the same pos tag to the new list of pos_tags `n_subwords` times
        pos_tags.extend([pos]*n_subwords)
        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, pos_tags, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_align_labels(sent,p_tags, n_tags)
    for sent, p_tags, n_tags in zip(sentences, pos_tags, ner_tags)
]

In [ ]:
tokenized_texts_final = [token_label_triple[0] for token_label_triple in tokenized_texts_and_labels]
pos_tags_final = [token_label_triple[1] for token_label_triple in tokenized_texts_and_labels]
labels = [token_label_triple[2] for token_label_triple in tokenized_texts_and_labels]


In [ ]:
print(len(tokenized_texts_final))
print(len(labels))

In [ ]:
#cut and pad to the desied length 75 bcz ab no of token increase ho gya
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_final],
                          maxlen=MAX_LENGTH, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
label_encoding_dict.keys()

In [ ]:
tags = pad_sequences([[label_encoding_dict["O"] if l not in label_encoding_dict else label_encoding_dict.get(l) for l in lab] for lab in labels], maxlen=MAX_LENGTH, dtype="long", value=label_encoding_dict["PAD"],
                          truncating="post", padding="post")

In [ ]:
#attenation mask to ignore PAD token
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
#10per train and validate
from sklearn.model_selection import train_test_split
"""
input_ids = [sent_dict["input_ids"] for sent_dict in tokenized_texts_and_labels]
tags = [sent_dict["labels"] for sent_dict in tokenized_texts_and_labels]
attention_masks = [sent_dict["attention_mask"] for sent_dict in tokenized_texts_and_labels]
"""
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2022, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2022, test_size=0.1)

In [ ]:
# convert to torch tensors
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
#training time shuffling of the data and testing time we pass them sequentially
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE)

In [ ]:
#load a pre-trained BERT model - transfer learning
bert_model = AMTC.from_pretrained(MODEL, return_dict=True)

In [ ]:
#loss
criterion = nn.BCELoss()

In [ ]:
#classifier layer
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         outputs = torch.sigmoid(self.linear(x))
         return outputs

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_metric
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer

In [ ]:

model_checkpoint = "distilbert-base-uncased"
model = AMTC.from_pretrained(model_checkpoint, num_labels=len(LABELS))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=1e-5,
)

#data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[LABELS[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[LABELS[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('drug-disease-ner.model')

In [ ]:
from transformers import BertForTokenClassification
bert_model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(LABELS),
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
#device
device = "cpu"
ON_CPU = True
if ON_CPU:
  device = "cpu"
else:
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#push the parameters to the default device
bert_model.to(device)

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(bert_model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(bert_model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = Adam(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
## Store the average loss after each epoch so we can plot them.
train_loss_values, validation_loss_values = [], []

for _ in tqdm(range(NUM_EPOCHS)):
    bert_model.zero_grad()
    # Put the model into training mode.
    bert_model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Before Back pass - clear grads
        bert_model.zero_grad()

        #DOES NOT LOOK LIKE A DIMENSION ERROR HERE
        print(len(b_input_ids))
        print(len(b_labels))

        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        output = bert_model(b_input_ids, attention_mask=b_input_mask)
        classifier = LogisticRegression(bert_model.config.hidden_size, len(LABELS)-1)
        output = classifier(output.pooler_output)
        #our activation function
        output = torch.sigmoid(output)    
        loss = 0
        if b_labels is not None:
            loss = criterion(output, b_labels)
        #loss
        #loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    train_loss_values.append(avg_train_loss)

    print()

#Pytorch Lightning

In [ ]:
"""
 tokenized_inputs = tokenizer.encode_plus(list(examples["tokens"]), truncation=True, is_split_into_words=True, max_length=MAX_LENGTH,
      return_token_type_ids=False,
      padding="max_length",
      return_attention_mask=True,
      return_tensors='pt')
    labels_t = []
    pos_t = []



   
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels_t.append(label_ids)
    for i, pos in enumerate(examples["pos"]):
        words_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        pos_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                pos_ids.append(-100)
            elif label[word_idx] == '0':
                pos_ids.append(0)
            elif word_idx != previous_word_idx:
                pos_ids.append(pos_encoding_dict[pos[word_idx]])
            else:
                pos_ids.append(pos_encoding_dict[pos[word_idx]] if pos_all_tokens else -100)
            previous_word_idx = word_idx    
        pos_t.append(pos_ids)
"""

In [ ]:
#creating a pytorch dataset class
class MedicalDataset(Dataset):
  def __init__(self, data, tokenizer):
    super().__init__()
    self.tokenizer = tokenizer
    self.data = data
    self.dataLen = len(self.data)

  def __len__(self):
    return self.dataLen

  def tokenize_and_align_labels(self, examples):
    label_all_tokens = True
    pos_all_tokens = True

    tokenized_inputs = tokenizer.encode_plus(list(examples["tokens"]), truncation=True, is_split_into_words=True, max_length=MAX_LENGTH,
      return_token_type_ids=False,
      padding="max_length",
      return_attention_mask=True,
      return_tensors='pt')
    
    labels = list(examples["tokens"])
    pos = list(examples["pos"])

    labels_t = pad_sequences([label_encoding_dict.get(lab) for lab in labels],
                     maxlen=MAX_LENGTH, value=label_encoding_dict["PAD"], padding="post",
                     dtype="long", truncating="post")
    
    pos_t = pad_sequences([pos_encoding_dict.get(pt) for pt in pos],
                     maxlen=MAX_LENGTH, value=pos_encoding_dict["PAD"], padding="post",
                     dtype="long", truncating="post")
    return dict(
      input_ids=tokenized_inputs["input_ids"].flatten(),
      attention_mask=tokenized_inputs["attention_mask"].flatten(),
      pos = torch.FloatTensor(pos_t),
      labels=torch.FloatTensor(labels_t)
    )

  def __getitem__(self, index: int):
    data_rows = self.data[self.data['sent_ids'] == i]
    return self.tokenize_and_align_labels(data_rows)

In [ ]:
class MedicalTextDataModule(pl.LightningDataModule):
  #three datasets - training, testing, validation dataset
  def __init__(self, train_df, val_df, test_df, tokenizer):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.val_df = val_df
    self.tokenizer = tokenizer

  #datasets setup - training, validation, testing  
  def setup(self, stage=None):
    self.train_dataset = MedicalDataset(
      self.train_df,
      self.tokenizer
    )
    self.val_dataset = MedicalDataset(
        self.val_df, 
        self.tokenizer)
    
    self.test_dataset = MedicalDataset(
      self.test_df,
      self.tokenizer
    )

  #The num_workers attribute tells the data loader instance how many sub-processes to use for data loading.
  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size= BATCH_SIZE,
      shuffle=True,
      num_workers=2)
    
  def val_dataloader(self):
    return DataLoader(
      self.val_dataset,
      batch_size= BATCH_SIZE,
      shuffle=False,
      num_workers=2)

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=BATCH_SIZE,
      num_workers=2)

In [ ]:
#load a pre-trained BERT model - transfer learning
bert_model = AMTC.from_pretrained(MODEL, return_dict=True)

In [ ]:
class MedicalTextLabelClassifier(pl.LightningModule):

  def __init__(self, bert_model, n_classes, criterion, n_training_steps=None):
    super().__init__()
    #pre-trained bert_model
    self.bert = bert_model
    #top logistic regression layer
    self.classifier = LogisticRegression(self.bert.config.hidden_size, n_classes)
    #how many times step is called
    self.n_training_steps = n_training_steps
    self.criterion = criterion

  #the forward function enables us to define how the model goes from input to output
  def forward(self, input_ids, attention_mask, labels=None):
    #reset the weights before backward pass
    self.bert.zero_grad()
    output = self.bert(input_ids, attention_mask=attention_mask, labels = labels)
    #last layer to a classifier output
    #output = self.classifier(output.pooler_output)
    #our activation function
    output = torch.sigmoid(output)    
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    #attention masks - binary masks to hide the padded indices
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    if batch_idx % 5000:
      print({"loss": loss})
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    return loss

  def training_epoch_end(self, outputs):
    #collecting the labels and predictions from the output
    labels = []
    predictions = []
    for output in outputs:
      for output_labels in output["labels"].detach().cpu():
        labels.append(output_labels)
      for output_predictions in output["predictions"].detach().cpu():
        predictions.append(output_predictions)

    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)

  def config_params(self, fineTune=False):
    if fineTune:
      param_optimizer = list(self.bert.named_parameters())
      no_decay = ['bias', 'gamma', 'beta']
      optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.01},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.0}
      ]
    else:
        param_optimizer = list(self.bert.named_parameters())
        optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
    return optimizer_grouped_parameters
  
  def configure_optimizers(self):
    opt_params = self.config_params(True)

    optimizer = Adam(opt_params, lr=1e-5)

    #the scheduler enables us to adjust the learning rate
    scheduler = ExponentialLR(optimizer, gamma=0.9)

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [ ]:
#criterion - measuring the Binary Cross Entropy between the target and the input probabilities
criterion = nn.BCELoss()

In [ ]:
#training
#calculating number of train steps
total_training_steps = BATCH_SIZE * NUM_EPOCHS

model = MedicalTextLabelClassifier(
    bert_model,
  n_classes=len(LABELS)-1,
  criterion = criterion,
  n_training_steps=total_training_steps
)

#checkpointing and saving

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    filename="sample-NER-{epoch:02d}-{val_loss:.2f}",
    save_top_k=2,
    mode="min",
)


#lightning module training configs
trainer = pl.Trainer(
  logger= None,
  checkpoint_callback= None,
  callbacks= None,
  gpus=1,
  max_epochs=NUM_EPOCHS
)

In [ ]:
#create a data module with all datasets in place
med_data_module = MedicalTextDataModule(traindf, valdf, valdf, tokenizer)

In [ ]:
#fitting 
trainer.fit(model, med_data_module)

In [ ]:
#evaluation mode
model.eval()
model.freeze()

In [ ]:
device = torch.device('cpu')
trained_model = model.to(device)
#evaluation
val_dataset = MedicalDataset(valdf, tokenizer)

predictions = []
labels = []

#wrap the iterable with tqdm
for item in tqdm(val_dataset):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())

predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

In [ ]:
CUTOFF = 0.5
#fscore, precision, recall, etc
y_pred = predictions.numpy()

#true labels, vs predictions - assigned an upper and lower bound
y_true = labels.numpy()
y_pred = np.where(y_pred >= CUTOFF, 1, 0)


In [ ]:
classification_report(y_true, y_pred)

In [ ]:
def getSavedModel(path):
  model = MedicalTextLabelClassifier.load_from_checkpoint(path)
  model.eval()
  return model